# Final Project

### Files and imports 

In [1]:
import pandas as pd
import glob
import utils
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

import shap
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

DATA = "./csv/"

In [2]:
# ese *1.csv solo pilla los originales porque los tratados acaban en t.csv
files = glob.glob(f"{DATA}*1.csv")

In [3]:
files.sort()

In [4]:
files

['./csv/02_197706_1.csv',
 './csv/02_197903_1.csv',
 './csv/02_198210_1.csv',
 './csv/02_198606_1.csv',
 './csv/02_198910_1.csv',
 './csv/02_199306_1.csv',
 './csv/02_199603_1.csv',
 './csv/02_200003_1.csv',
 './csv/02_200403_1.csv',
 './csv/02_200803_1.csv',
 './csv/02_201111_1.csv',
 './csv/02_201512_1.csv',
 './csv/02_201606_1.csv',
 './csv/02_201904_1.csv',
 './csv/02_201911_1.csv']

#### Correct the files

In [5]:
for file in files:

    test = pd.read_csv(file)
    
    # normaliza nombres de municipio y provincia
    test["Nombre de Municipio"] = test["Nombre de Municipio"].apply(utils.proper)
    test["Nombre de Provincia"] = test["Nombre de Provincia"].apply(utils.proper)

    # se queda solo con las siglas del partido (normalizado)
    test.rename(columns={column: utils.acronym(column) for column in test.columns[13:]}, inplace=True)
    
    test = test.rename(columns={'PODEMOS-IU-EQUO': 'PODEMOS', 'PODEMOS-IU': 'PODEMOS', 'PODEMOS-COM': 'PODEMOS', 
                                'PODEMOS-COMPROMÍS-EUPV': 'PODEMOS', 'PODEMOS-EN': 'PODEMOS', 'PODEMOS-EN MAREA-ANOVA-EU': 
                                'PODEMOS', 'EN COMÚ': 'PODEMOS', 'CCA-PNC-NC': 'CCA-PNC', 'COMPROMÍS 2019': 'MÉS COMPROMÍS', 
                                "C'S": 'CS', 'ECP-GUANYEM EL CANVI': 'PODEMOS', 'NCA': 'CCA-PNC', 'CC-NC-PNC': 'CCA-PNC', 
                                'AMAIUR': 'EH BILDU', 'CDC': 'CIU', 'ERC-SOBIRANISTES': 'ERC', 'ERC-CATSÍ': 'ERC', 
                                'ERC-CATSI': 'ERC', 'ESQUERRA': 'ERC', 'ECP': 'PODEMOS', 'P.S.O.E.': 'PSOE', 'PSOE-PROGR.': 
                                'PSOE', 'P.P.': 'PP'})
    
    # calcula el partido con mas votos, cuantos votos y lo guarda
    utils.compute_popular(test).to_csv(f"{file[:-4]}_treat.csv")

In [6]:
# test.columns[:13] common vote, town info. Dropping this gives list of political parties
test.drop(columns=test.columns[:13]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8131 entries, 0 to 8130
Data columns (total 69 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   PSOE                       8131 non-null   int64 
 1   PP                         8131 non-null   int64 
 2   VOX                        8131 non-null   int64 
 3   PODEMOS                    8131 non-null   int64 
 4   ERC                        8131 non-null   int64 
 5   CS                         8131 non-null   int64 
 6   JXCAT-JUNTS                8131 non-null   int64 
 7   PODEMOS                    8131 non-null   int64 
 8   EAJ-PNV                    8131 non-null   int64 
 9   EH BILDU                   8131 non-null   int64 
 10  MÁS PAÍS-EQUO              8131 non-null   int64 
 11  CUP-PR                     8131 non-null   int64 
 12  PODEMOS-EU                 8131 non-null   int64 
 13  CCA-PNC                    8131 non-null   int64 
 14  NA+     

#### Get elections from 2000 until now

In [7]:
files_treated = glob.glob(f"{DATA}02_20*t.csv")
common_columns = ["Unnamed: 0", "Nombre de Comunidad", "Código de Provincia", "Nombre de Provincia", "Código de Municipio",
                  "Nombre de Municipio"]
char_columns = ["Unnamed: 0", "Nombre de Comunidad", "Código de Provincia", "Nombre de Provincia", "Código de Municipio",
                   "Nombre de Municipio", "Población", "Número de mesas", "Popular"]
elections20_ = pd.read_csv(files_treated[0])
elections20_ = elections20_.reset_index(drop=True)
for col in elections20_.columns:
    if col not in char_columns:
        elections20_[col] = elections20_[col]/elections20_["Población"]
elections20_ = elections20_.drop(columns=["Población","Unnamed: 0"])
for col in elections20_.columns:
    if col not in common_columns:
        elections20_.rename(columns={col: col+"_2000_03"},inplace=True) 
for file in files_treated[1:]:
    ele = pd.read_csv(file)
    ele.rename(columns={"Población ": "Población"},inplace=True) 
    ele = ele.reset_index(drop=True)
    name = '_'+file.split('_')[1][:4]+'_'+file.split('_')[1][4:]
    for col in ele.columns:
        if col not in char_columns:
            ele[col] = ele[col]/ele["Población"]
    for col in ele.columns:
        if col not in common_columns:
            ele.rename(columns={col: col+f'{name}'},inplace=True) 
    elections20_ = pd.merge(elections20_, ele, on=["Nombre de Comunidad", "Código de Provincia",
                                                     "Nombre de Provincia", "Código de Municipio",
                                                     "Nombre de Municipio"],how = 'outer',suffixes=("",f'{name}'))
    elections20_ = elections20_.drop(columns=["Población"+f'{name}',"Unnamed: 0"])

### Analysis of elections from 2010 until now

In [8]:
p1111 = elections20_["Popular_2011_11"].value_counts()
p1215 = elections20_["Popular_2015_12"].value_counts()
p0616 = elections20_["Popular_2016_06"].value_counts()
p0419 = elections20_["Popular_2019_04"].value_counts()
p1119 = elections20_["Popular_2019_11"].value_counts()
popular201_ = p1111.to_frame().join(p1215, how='outer').join(p0616, how='outer').join(p0419, how='outer').join(p1119, 
                    how='outer').sort_values(by=['Popular_2019_11', 'Popular_2019_04'], ascending=False)
popular201_ = popular201_.replace(np.nan,0.0)
popular201_.head(15)

KeyError: 'Popular_2011_11'

#### Predicting elections on nov 2019

In [ ]:
X = popular201_.copy().drop(columns='Popular_2019_11')
y = popular201_['Popular_2019_11']
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state = 0)

linreg_model = LinearRegression()
linreg_model.fit(Xtrain, ytrain)
prediction_linreg = linreg_model.predict(Xtest)

dtree = tree.DecisionTreeRegressor()
dtree.fit(Xtrain, ytrain)
prediction_dtree = dtree.predict(Xtest)

rforest = RandomForestRegressor()
rforest.fit(Xtrain, ytrain)
prediction_rforest = rforest.predict(Xtest)

nn = MLPRegressor(solver='lbfgs', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state = 1)
nn.fit(Xtrain, ytrain)
prediction_nn = nn.predict(Xtest)

results201_ = ytest.to_frame()
results201_['Prediciton_linreg'] = prediction_linreg.astype(int).astype(float)
results201_['Prediciton_dtree'] = prediction_dtree.astype(int).astype(float)
results201_['Prediciton_rforest'] = prediction_rforest.astype(int).astype(float)
results201_['Prediciton_nn'] = prediction_nn.astype(int).astype(float)

results201_

In [ ]:
mse_linreg = mean_squared_error(ytest, prediction_linreg)
mse_dtree = mean_squared_error(ytest, prediction_dtree)
mse_rforest = mean_squared_error(ytest, prediction_rforest)
mse_nn = mean_squared_error(ytest, prediction_nn)

mse_results = {'mse_linreg': mse_linreg, 'mse_dtree': mse_dtree, 'mse_rforest': mse_rforest, 'mse_nn': mse_nn}
mse_results

#### Analysis of linear regression of predicting elections on nov 2019

In [ ]:
fig, axs = plt.subplots()
plt.scatter(Xtest['Popular_2019_04'], prediction_linreg)
plt.plot([0,1],[0,1], transform=axs.transAxes)
plt.xlabel('Popular_2019_04')
plt.ylabel('Popular_2019_11')
plt.show()

#### Analysis of NN of predicting elections on nov 2019

In [ ]:
# Use Shap explainer to interpret values in the test set:
shap.initjs() 
explainer = shap.KernelExplainer(nn.predict, Xtrain)
shap_values = explainer.shap_values(X)

In [ ]:
shap.summary_plot(shap_values, X, matplotlib=True, show=False)
plt.savefig(f"shap_figures/summary_plot.png", bbox_inches="tight")
plt.clf()

In [ ]:
shap.dependence_plot("Popular_2019_04", shap_values, X, matplotlib=True, show=False)
plt.savefig(f"shap_figures/dependence_plot.png", bbox_inches="tight")
plt.clf()

### Analysis of elections from 2000 until now

In [ ]:
p0300 = elections20_["Popular_2000_03"].value_counts().to_frame()
p0304 = elections20_["Popular_2004_03"].value_counts()
p0308 = elections20_["Popular_2008_03"].value_counts()
popular20_ = p0300.join(p0304, how='outer').join(p0308, how='outer').join(p1111, how='outer').join(p1215, 
                    how='outer').join(p0616, how='outer').join(p0419, how='outer').join(p1119, 
                    how='outer').sort_values(by=['Popular_2019_11', 'Popular_2019_04'], ascending=False)
popular20_ = popular20_.replace(np.nan,0.0)
popular20_.head(15)

#### Predicting elections on nov 2019

In [ ]:
X = popular20_.copy().drop(columns='Popular_2019_11')
y = popular20_['Popular_2019_11']
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state = 0)

linreg_model = LinearRegression()
linreg_model.fit(Xtrain, ytrain)
prediction_linreg = linreg_model.predict(Xtest)

dtree = tree.DecisionTreeRegressor()
dtree.fit(Xtrain, ytrain)
prediction_dtree = dtree.predict(Xtest)

rforest = RandomForestRegressor()
rforest.fit(Xtrain, ytrain)
prediction_rforest = rforest.predict(Xtest)

nn = MLPRegressor(solver='lbfgs', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state = 18)
nn.fit(Xtrain, ytrain)
prediction_nn = nn.predict(Xtest)

results20_ = ytest.to_frame()
results20_['Prediciton_linreg'] = prediction_linreg.astype(int).astype(float)
results20_['Prediciton_dtree'] = prediction_dtree.astype(int).astype(float)
results20_['Prediciton_rforest'] = prediction_rforest.astype(int).astype(float)
results20_['Prediciton_nn'] = prediction_nn.astype(int).astype(float)

results20_

In [ ]:
mse_linreg = mean_squared_error(ytest, prediction_linreg)
mse_dtree = mean_squared_error(ytest, prediction_dtree)
mse_rforest = mean_squared_error(ytest, prediction_rforest)
mse_nn = mean_squared_error(ytest, prediction_nn)

mse_results = {'mse_linreg': mse_linreg, 'mse_dtree': mse_dtree, 'mse_rforest': mse_rforest, 'mse_nn': mse_nn}
mse_results

#### Analysis of linear regression of predicting elections on nov 2019

In [ ]:
fig, axs = plt.subplots()
plt.scatter(Xtest['Popular_2019_04'], prediction_linreg)
plt.plot([0,1],[0,1], transform=axs.transAxes)
plt.xlabel('Popular_2019_04')
plt.ylabel('Predicted_2019_11')
plt.show()

#### Analysis of NN of predicting elections on nov 2019

In [ ]:
# Use Shap explainer to interpret values in the test set:
shap.initjs() 
explainer = shap.KernelExplainer(nn.predict, Xtrain)
shap_values = explainer.shap_values(X)

In [ ]:
shap.summary_plot(shap_values, X, matplotlib=True, show=False)
plt.savefig(f"shap_figures/summary_plot_2.png", bbox_inches="tight")
plt.clf()


In [ ]:
shap.dependence_plot("Popular_2019_04", shap_values, X, matplotlib=True, show=False)
plt.savefig(f"shap_figures/dependence_plot_2.png", bbox_inches="tight")
plt.clf()

### Analysis of common political parties in 2019

In [ ]:
files_treated2 = glob.glob(f"{DATA}02_2019*t.csv")
ele1 = pd.read_csv(files_treated2[0])
ele1 = ele1.reset_index(drop=True)
ele2 = pd.read_csv(files_treated2[1])
ele2 = ele2.reset_index(drop=True)
elections2019 = pd.merge(ele1, ele2, on=["Unnamed: 0", "Nombre de Comunidad","Código de Provincia","Nombre de Provincia",
                                     "Código de Municipio","Nombre de Municipio", "Población"],suffixes=("_apr","_nov"))

In [ ]:
elections2019['Número de mesas_apr'].apply(lambda x: float(x))
elections2019['Número de mesas_nov'].apply(lambda x: float(x))
elections2019['Total censo electoral_apr'].apply(lambda x: float(x))
elections2019['Total censo electoral_nov'].apply(lambda x: float(x))
elections2019['Popular votos_apr'].apply(lambda x: float(x))
elections2019['Popular votos_nov'].apply(lambda x: float(x))
elections2019 = elections2019.replace(np.nan,0.0)

In [ ]:
elections2019.info()

In [ ]:
for col in elections2019.columns:
    if col not in ["Unnamed: 0", "Nombre de Comunidad", "Código de Provincia", "Nombre de Provincia", "Código de Municipio",
                   "Nombre de Municipio", "Población", "Número de mesas_apr", "Número de mesas_nov", "Popular_apr", 
                   "Popular_nov"]:
        elections2019[col] = elections2019[col]/elections2019["Población"]

In [ ]:
elections2019["Popular_apr"].value_counts().to_frame().join(elections2019["Popular_nov"].value_counts(), how='outer').sort_values(by=['Popular_nov', 'Popular_apr'], ascending=False)

In [ ]:
fig, axs = plt.subplots(1, 3, sharex=False, figsize=(15,5))
fig.suptitle('PSOE')
sns.histplot(ax=axs[0], x=elections2019['PSOE_apr'].values, kde=True).set_title('april')
sns.histplot(ax=axs[1], x=elections2019['PSOE_nov'].values, kde=True).set_title('november')
sns.histplot(ax=axs[2], x=elections2019['PSOE_apr'].values, kde=True)
sns.histplot(ax=axs[2], x=elections2019['PSOE_nov'].values, kde=True).set_title('PSOE april vs november')
plt.legend(['april','november'])
axs[0].set_ylim(0, 500)
axs[1].set_ylim(0, 500)
axs[2].set_ylim(0, 500)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, sharex=False, figsize=(15,5))
fig.suptitle('PP')
sns.histplot(ax=axs[0], x=elections2019['PP_apr'].values, kde=True).set_title('april')
sns.histplot(ax=axs[1], x=elections2019['PP_nov'].values, kde=True).set_title('november')
sns.histplot(ax=axs[2], x=elections2019['PP_apr'].values, kde=True)
sns.histplot(ax=axs[2], x=elections2019['PP_nov'].values, kde=True).set_title('PP april vs november')
plt.legend(['april','november'])
axs[0].set_ylim(0, 800)
axs[1].set_ylim(0, 800)
axs[2].set_ylim(0, 800)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, sharex=False, figsize=(15,5))
fig.suptitle('VOX')
sns.histplot(ax=axs[0], x=elections2019['VOX_apr'].values, kde=True).set_title('april')
sns.histplot(ax=axs[1], x=elections2019['VOX_nov'].values, kde=True).set_title('november')
sns.histplot(ax=axs[2], x=elections2019['VOX_apr'].values, kde=True)
sns.histplot(ax=axs[2], x=elections2019['VOX_nov'].values, kde=True).set_title('VOX april vs november')
plt.legend(['april','november'])
axs[0].set_ylim(0, 500)
axs[1].set_ylim(0, 500)
axs[2].set_ylim(0, 500)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, sharex=False, figsize=(15,5))
fig.suptitle('PODEMOS')
sns.histplot(ax=axs[0], x=elections2019['PODEMOS_apr'].values, kde=True).set_title('april')
sns.histplot(ax=axs[1], x=elections2019['PODEMOS_nov'].values, kde=True).set_title('november')
sns.histplot(ax=axs[2], x=elections2019['PODEMOS_apr'].values, kde=True)
sns.histplot(ax=axs[2], x=elections2019['PODEMOS_nov'].values, kde=True).set_title('PODEMOS april vs november')
plt.legend(['april','november'])
axs[0].set_ylim(0, 500)
axs[1].set_ylim(0, 500)
axs[2].set_ylim(0, 500)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, sharex=False, figsize=(15,5))
fig.suptitle('CS')
sns.histplot(ax=axs[0], x=elections2019['CS_apr'].values, kde=True).set_title('april')
sns.histplot(ax=axs[1], x=elections2019['CS_nov'].values, kde=True).set_title('november')
sns.histplot(ax=axs[2], x=elections2019['CS_apr'].values, kde=True)
sns.histplot(ax=axs[2], x=elections2019['CS_nov'].values, kde=True).set_title('CS april vs november')
plt.legend(['april','november'])
axs[0].set_ylim(0, 500)
axs[1].set_ylim(0, 500)
axs[2].set_ylim(0, 500)
plt.show()